In [182]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random
using Plots
include("quadruped.jl")
include("quadratic_cost.jl")
include("moi.jl")

  Activating environment at `C:\Users\pnadan\Documents\GitHub\Optimal-Control-Project\Project.toml`
Precompiling project...
  ✓ OpenSSL_jll
  ✓ ChainRulesCore
  ✓ ArrayInterface
  ✓ StructArrays
  ✓ WebIO
  ✓ FFMPEG_jll
  ✓ ChangesOfVariables
  ✓ FiniteDiff
  ✓ Interpolations
  ✓ FFMPEG
  ✓ JSExpr
  ✓ OSQP
  ✓ LogExpFunctions
  ✓ Blink
  ✓ GeometryBasics
  ✓ SpecialFunctions
  ✓ DiffRules
  ✓ MeshIO
  ✓ MeshCat
  ✓ ForwardDiff
  ✓ SparseDiffTools
  ✓ RobotDynamics
  ✓ RobotZoo
  ✓ TrajectoryOptimization
  ✓ Altro
  ✓ TrajOptPlots
  26 dependencies successfully precompiled in 49 seconds (136 already precompiled)
  23 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions


LoadError: LoadError: ArgumentError: Package MeshCatMechanisms not found in current path:
- Run `import Pkg; Pkg.add("MeshCatMechanisms")` to install the MeshCatMechanisms package.

in expression starting at C:\Users\pnadan\Documents\GitHub\Optimal-Control-Project\src\quadruped.jl:3

## Load Full Model

In [185]:
full_model = UnitreeA1()
n,m = state_dim(full_model), control_dim(full_model)
mvis = initialize_visualizer(full_model);

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8708
└ @ MeshCat C:\Users\pnadan\.julia\packages\MeshCat\Ax8pH\src\visualizer.jl:73


## Load Simplified Model

In [171]:
include("simple_quadruped.jl")

mf = RigidBodyDynamics.spatial_inertia(findbody(full_model.mech, "RR_foot")).mass
mb = RigidBodyDynamics.spatial_inertia(findbody(full_model.mech, "trunk")).mass + 
     RigidBodyDynamics.spatial_inertia(findbody(full_model.mech, "RR_hip")).mass*4 +
     RigidBodyDynamics.spatial_inertia(findbody(full_model.mech, "RR_thigh")).mass*4 +
     RigidBodyDynamics.spatial_inertia(findbody(full_model.mech, "RR_calf")).mass*4
J = RigidBodyDynamics.spatial_inertia(findbody(full_model.mech, "trunk")).moment
state = MechanismState(full_model.mech)
zero!(state)
leg = translation(relative_transform(state, 
      default_frame(findbody(full_model.mech, "RR_hip")),
      default_frame(findbody(full_model.mech, "RR_foot"))))
s1 = translation(relative_transform(state, 
     default_frame(findbody(full_model.mech, "trunk")),
     default_frame(findbody(full_model.mech, "RR_hip"))))
s1 = vcat(-s1[2], s1[1], s1[3])
s2 = s1.*[1,-1,1]
s3 = s1.*[-1,1,1]
s4 = s1.*[-1,-1,1]

model = SimpleQuadruped(mf=mf, mb=mb, Jx=J[1,1], Jy=J[2,2], Jz=J[3,3], ℓmax=leg[3], s1=s1, s2=s2, s3=s3, s4=s4)
n,m = state_dim(model), control_dim(model);

## Reference Trajectory & Contact Sequence

In [184]:
include("ref_trajectory.jl")
include("gait.jl")

# Discretization
tf = 4.4
dt = 0.1
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N);

# Reference Trajectory
Xref,Uref = reference_trajectory(model, times)

# Contact Sequence
contact_sched = contact_sequence_trot(times)

contact_sequence_trot (generic function with 1 method)

## Non-Linear Program for IPOPT

In [180]:
include("nlp.jl")
include("nlp_cost.jl")
include("nlp_constraint.jl")

LoadError: LoadError: UndefVarError: QuadraticCost not defined
in expression starting at C:\Users\pnadan\Documents\GitHub\Optimal-Control-Project\src\nlp.jl:1